In [2]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import os, sys
%matplotlib inline

In [5]:
os.getcwd()
os.chdir('C:\\Users\\Ben\\Documents\\682_project')

In [6]:
with open('data/self_play/memory.pickle', 'rb') as f:
    mem = pickle.load(f)

In [9]:
actions, states, reward = mem[0]

In [11]:
print(states[-1])

Radiant Bans:
16           Bloodseeker
66          Chaos Knight
77        Spirit Breaker
78      Treant Protector
88     Centaur Warrunner
115           Juggernaut
Name: localized_name, dtype: object
Dire Bans
5     Phantom Lancer
14             Viper
17    Crystal Maiden
24        Venomancer
84                Io
86            Medusa
Name: localized_name, dtype: object
Radiant
36          Tinker
46     Lifestealer
79       Ogre Magi
111        Undying
112      Anti-Mage
Name: localized_name, dtype: object
Dire
30            Pudge
33     Storm Spirit
35             Riki
71         Batrider
117            None
Name: localized_name, dtype: object


In [12]:
states[-1].radiant

array([86.,  3., 35., 55., 85.])

In [13]:
states[-1].dire

array([33., 19., 16., 66.,  2.])

In [14]:
actions

[6,
 14,
 72,
 48,
 10,
 41,
 86,
 33,
 3,
 19,
 82,
 95,
 97,
 92,
 35,
 16,
 55,
 66,
 84,
 7,
 85,
 69]